In [1]:
import sys
sys.path.append(r"C:\Users\bonda\Documents\Bondan\TEWS Testing\skydrifter")
import skydrifter as sd
from skydrifter.Utils.partisan import *
from skydrifter.Utils.nonpartisan import *

ModuleNotFoundError: No module named 'skydrifter'

In [ ]:
from skydrifter.PlotModeCollection.plotting import *

# Domestic Function

In [ ]:
def make_group_barplot_multiple_bar(df_perform,column,title,step,min_value,max_value,**kwargs):
    warm_colors = [
        '#2a81ff',
        '#E63946',  # Bright red
        '#FF9F1C',  # Golden yellow-orange
        'fa635cc',
        '#457B9D',  # Warm blue
        '#1D3557',  # Deep warm blue
        '#F77F00',  # Orange
        '#F4A261',  # Muted orange
        '#FFD166',  # Warm yellow
        '#D62828'] # Deep red
    df_combined = pd.DataFrame()
    for loop in range(0,len(column)):
        distance = df_perform[column[loop]]
        # Make Data Frame
        dd = copy.deepcopy(distance)
        dd.index = [i for i in range(0,len(dd.index))]
        df_temp = pd.DataFrame()
        df_temp['Error'] = [float(dd[i]) for i in range(0,len(dd))]
        # Make Range
        count = 0
        max_loop = min_value
        range_value = [min_value]
        while max_loop < max_value:
            max_loop = max_loop + step
            range_value.append(round(max_loop,2))
        # Group Data
        frequency = []
        frequency_label = []
        for i in range(0,len(range_value)-1):
            df_temp = df_temp[df_temp['Error'] >= range_value[i]]
            frequency.append(len(df_temp[df_temp['Error'] < range_value[i+1]]))
            frequency_label.append(str(range_value[i]) + '-' + str(range_value[i+1]))
        frequency.append(len(df_temp[df_temp['Error'] >= max_value]))
        frequency_label.append('> ' + str(max_value))
        df_freq = pd.DataFrame()
        df_freq['Range Value'] = frequency_label + ['Unidentified']
        df_freq['Frequency'] = frequency + [len(df_temp[df_temp['Error'] == -1])]
        df_freq['Source'] = [column[loop] for i in range(0,len(df_freq))]
        #
        df_combined = pd.concat([df_combined,df_freq])
    # Plot grouped bar plot
    plt.figure(figsize=(8, 6))
    sns.barplot(data=df_combined, x='Range Value', y='Frequency', hue='Source', palette=[warm_colors[color] for color in range(0,len(column))])
    plt.xticks(df_combined['Range Value'], rotation=90)
    plt.xlabel('Groups')
    plt.ylabel('Frequency')
    plt.title(title)
    plt.legend(title='Source')
    # kwargs save
    try:
        plt.savefig(kwargs['target_folder'],bbox_inches='tight',dpi=1500)
    except:
        pass
    plt.show()

In [ ]:
def precision_report(TP,TN,FP,FN):
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (TP + TN) / sum([TP,TN,FP,FN])
    return precision

def recall_report(TP,TN,FP,FN):
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (TP + TN) / sum([TP,TN,FP,FN])
    return recall

def accuracy_report(TP,TN,FP,FN):
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (TP + TN) / sum([TP,TN,FP,FN])
    return accuracy

def f1_report(TP,TN,FP,FN):
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (TP + TN) / sum([TP,TN,FP,FN])
    return f1

In [ ]:
def make_row_assessment(df_perform,row_name):
    df_output = pd.DataFrame()
    # 1
    TP = df_perform['TP_p'].sum() + df_perform['TP_s'].sum()
    TN = df_perform['TN_p'].sum() + df_perform['TN_s'].sum()
    FP = df_perform['FP_p'].sum() + df_perform['FP_s'].sum()
    FN = df_perform['FN_p'].sum() + df_perform['FN_s'].sum()
    df_output['precision'] = [precision_report(TP,TN,FP,FN)]
    df_output['recall'] = [recall_report(TP,TN,FP,FN)]
    df_output['accuracy'] = [accuracy_report(TP,TN,FP,FN)]
    df_output['f1'] = [f1_report(TP,TN,FP,FN)]
    df_output['mean'] = [np.mean(df_perform['residual_p'].tolist() + df_perform['residual_s'].tolist())]
    df_output['median'] = [np.median(df_perform['residual_p'].tolist() + df_perform['residual_s'].tolist())]
    df_output['q1'] = [np.quantile(df_perform['residual_p'].tolist() + df_perform['residual_s'].tolist(),0.25)]
    df_output['q3'] = [np.quantile(df_perform['residual_p'].tolist() + df_perform['residual_s'].tolist(),0.75)]
    # 2
    TP = df_perform['TP_p'].sum()
    TN = df_perform['TN_p'].sum()
    FP = df_perform['FP_p'].sum()
    FN = df_perform['FN_p'].sum()
    df_output['precision_p'] = [precision_report(TP,TN,FP,FN)]
    df_output['recall_p'] = [recall_report(TP,TN,FP,FN)]
    df_output['accuracy_p'] = [accuracy_report(TP,TN,FP,FN)]
    df_output['f1_p'] = [f1_report(TP,TN,FP,FN)]
    df_output['mean_p'] = [df_perform['residual_p'].mean()]
    df_output['median_p'] = [df_perform['residual_p'].median()]
    df_output['q1_p'] = [np.quantile(df_perform['residual_p'].to_numpy(),0.25)]
    df_output['q3_p'] = [np.quantile(df_perform['residual_p'].to_numpy(),0.75)]
    # 3
    TP = df_perform['TP_s'].sum()
    TN = df_perform['TN_s'].sum()
    FP = df_perform['FP_s'].sum()
    FN = df_perform['FN_s'].sum()
    df_output['precision_s'] = [precision_report(TP,TN,FP,FN)]
    df_output['recall_s'] = [recall_report(TP,TN,FP,FN)]
    df_output['accuracy_s'] = [accuracy_report(TP,TN,FP,FN)]
    df_output['f1_s'] = [f1_report(TP,TN,FP,FN)]
    df_output['mean_s'] = [df_perform['residual_s'].mean()]
    df_output['median_s'] = [df_perform['residual_s'].median()]
    df_output['q1_s'] = [np.quantile(df_perform['residual_s'].to_numpy(),0.25)]
    df_output['q3_s'] = [np.quantile(df_perform['residual_s'].to_numpy(),0.75)]
    # 4
    df_output.index = [row_name]
    return df_output

# Set Path

In [ ]:
import glob
all_model_path = glob.glob(r"C:\Users\bonda\Documents\Bondan\TEWS Testing\model_collection\model_collection\PhaseNet Seisbench Jan 2025\*")
for i in range(0,len(all_model_path)):
    print(i)
    print(all_model_path[i])

In [ ]:
dataset_path = r"C:\Users\bonda\Documents\Bondan\TEWS Testing\dataset\2025 Januray Picking Dataset 20 Hz"
model_path = all_model_path[8]

# Load Data

In [ ]:
model_metadata = pd.read_csv(join_by([model_path,'metadata',"model_metadata.csv"],separator='\\'))
model_metadata

In [ ]:
train_perform_before = pd.read_csv(join_by([model_path,'performance',"train_performance_before.csv"],separator='\\'))
test_perform_before = pd.read_csv(join_by([model_path,'performance',"test_performance_before.csv"],separator='\\'))

In [ ]:
train_perform_after = pd.read_csv(join_by([model_path,'performance',"train_performance_after.csv"],separator='\\'))
test_perform_after = pd.read_csv(join_by([model_path,'performance',"test_performance_after.csv"],separator='\\'))

In [ ]:
architecture = model_metadata[model_metadata['param'] == 'model']['value'].iloc[0]
pretrain = model_metadata[model_metadata['param'] == 'pretrain']['value'].iloc[0]
sr = model_metadata[model_metadata['param'] == 'sampling_rate']['value'].iloc[0]
wd = model_metadata[model_metadata['param'] == 'window_duration']['value'].iloc[0]
architecture,pretrain,sr,wd

# Train Data Residual and Probability Plot

### P-Arrival Residual Plot

In [ ]:
filename = '[1]_P_arrival_picking_time_residual_train_data_before.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P-Arrival Picking Time Residual\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nBefore Fine Tuning In Train Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(train_perform_before['residual_p'],title=title,target_folder=target_folder,step=1,min_value=0,max_value=20)

In [ ]:
filename = '[2]_P_arrival_picking_time_residual_train_data_after.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P-Arrival Picking Time Residual\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nAfter Fine Tuning In Train Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(train_perform_after['residual_p'],title=title,target_folder=target_folder,step=1,min_value=0,max_value=20)

### S-Arrival Residual Residual Plot

In [ ]:
filename = '[3]_S_arrival_picking_time_residual_train_data_before.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'S-Arrival Picking Time Residual\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nBefore Fine Tuning In Train Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(train_perform_before['residual_s'],target_folder=target_folder,title=title,step=1,min_value=0,max_value=20)

In [ ]:
filename = '[4]_S_arrival_picking_time_residual_train_data_after.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'S-Arrival Picking Time Residual\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nAfter Fine Tuning In Train Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(train_perform_after['residual_s'],target_folder=target_folder,title=title,step=1,min_value=0,max_value=20)

### Multipe Bar Residual Plot

In [ ]:
filename = '[5]_mixed_arrival_picking_time_residual_train_data_before.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P and S Arrival Picking Time Residual\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nBefore Fine Tuning In Train Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot_multiple_bar(df_perform=train_perform_before,target_folder=target_folder,column=['residual_p','residual_s'],title=title,step=1,min_value=0,max_value=20)

In [ ]:
filename = '[6]_mixed_arrival_picking_time_residual_train_data_after.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P and S Arrival Picking Time Residual\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nAfter Fine Tuning In Train Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot_multiple_bar(df_perform=train_perform_after,column=['residual_p','residual_s'],target_folder=target_folder,title=title,step=1,min_value=0,max_value=20)

### P-Arrival Probability Plot

In [ ]:
filename = '[7]_P_probability_distribution_train_data_before.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P-Arrival Picking Probability Distribution\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nBefore Fine Tuning In Train Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(train_perform_before['pred_prob_p'],target_folder=target_folder,title=title,step=0.05,min_value=0,max_value=1)

In [ ]:
filename = '[8]_P_probability_distribution_train_data_after.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P-Arrival Picking Probability Distribution\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nAfter Fine Tuning In Train Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(train_perform_after['pred_prob_p'],target_folder=target_folder,title=title,step=0.05,min_value=0,max_value=1)

### S-Arrival Probability Plot

In [ ]:
filename = '[9]_S_probability_distribution_train_data_before.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'S-Arrival Picking Probability Distribution\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nBefore Fine Tuning In Train Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(train_perform_before['pred_prob_s'],target_folder=target_folder,title=title,step=0.05,min_value=0,max_value=1)

In [ ]:
filename = '[10]_S_probability_distribution_train_data_after.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'S-Arrival Picking Probability Distribution\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nAfter Fine Tuning In Train Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(train_perform_after['pred_prob_s'],target_folder=target_folder,title=title,step=0.05,min_value=0,max_value=1)

### Multiple Bar Probability Plot

In [ ]:
filename = '[11]_mixed_probability_distribution_train_data_before.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P and S Arrival Picking Probability Distribution\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nBefore Fine Tuning In Train Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot_multiple_bar(df_perform=train_perform_before,target_folder=target_folder,column=['pred_prob_p','pred_prob_s'],title=title,step=0.05,min_value=0,max_value=1)

In [ ]:
filename = '[12]_mixed_probability_distribution_train_data_after.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P and S Arrival Picking Probability Distribution\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nAfter Fine Tuning In Train Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot_multiple_bar(df_perform=train_perform_after,target_folder=target_folder,column=['pred_prob_p','pred_prob_s'],title=title,step=0.05,min_value=0,max_value=1)

# Test Data Residual and Probability Plot

### P-Arrival Residual Plot

In [ ]:
filename = '[13]_S_arrival_picking_time_residual_test_data_before.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P-Arrival Picking Time Residual\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nBefore Fine Tuning In Test Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(test_perform_before['residual_p'],target_folder=target_folder,title=title,step=1,min_value=0,max_value=20)

In [ ]:
filename = '[14]_P_arrival_picking_time_residual_test_data_after.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P-Arrival Picking Time Residual\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nAfter Fine Tuning In Test Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(test_perform_after['residual_p'],target_folder=target_folder,title=title,step=1,min_value=0,max_value=20)

### S-Arrival Residual Plot

In [ ]:
filename = '[15]_S_arrival_picking_time_residual_test_data_before.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'S-Arrival Picking Time Residual\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nBefore Fine Tuning In Test Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(test_perform_before['residual_s'],target_folder=target_folder,title=title,step=1,min_value=0,max_value=20)

In [ ]:
filename = '[16]_S_arrival_picking_time_residual_test_data_after.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'S-Arrival Picking Time Residual\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nAfter Fine Tuning In Test Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(test_perform_after['residual_s'],target_folder=target_folder,title=title,step=1,min_value=0,max_value=20)

### Multipe Bar Residual Plot

In [ ]:
filename = '[17]_mixed_arrival_picking_time_residual_test_data_before.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P and S Arrival Picking Time Residual\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nBefore Fine Tuning In Test Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot_multiple_bar(df_perform=test_perform_before,target_folder=target_folder,column=['residual_p','residual_s'],title=title,step=1,min_value=0,max_value=20)

In [ ]:
filename = '[18]_mixed_arrival_picking_time_residual_test_data_after.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P and S Arrival Picking Time Residual\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nAfter Fine Tuning In Test Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot_multiple_bar(df_perform=test_perform_after,target_folder=target_folder,column=['residual_p','residual_s'],title=title,step=1,min_value=0,max_value=20)

### P-Arrival Probability Plot

In [ ]:
filename = '[19]_P_probability_distribution_test_data_before.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P-Arrival Picking Probability Distribution\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nBefore Fine Tuning In Test Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(test_perform_before['pred_prob_p'],target_folder=target_folder,title=title,step=0.05,min_value=0,max_value=1)

In [ ]:
filename = '[20]_P_probability_distribution_test_data_after.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P-Arrival Picking Probability Distribution\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nAfter Fine Tuning In Test Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(test_perform_after['pred_prob_p'],target_folder=target_folder,title=title,step=0.05,min_value=0,max_value=1)

### S-Arrival Probability Plot

In [ ]:
filename = '[21]_S_probability_distribution_test_data_before.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'S-Arrival Picking Probability Distribution\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nBefore Fine Tuning In Test Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(test_perform_before['pred_prob_s'],target_folder=target_folder,title=title,step=0.05,min_value=0,max_value=1)

In [ ]:
filename = '[21]_S_probability_distribution_test_data_after.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'S-Arrival Picking Probability Distribution\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nAfter Fine Tuning In Test Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot(test_perform_after['pred_prob_s'],target_folder=target_folder,title=title,step=0.05,min_value=0,max_value=1)

### Multiple Bar Probability Plot

In [ ]:
filename = '[23]_mixed_probability_distribution_test_data_before.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P and S Arrival Picking Probability Distribution\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nBefore Fine Tuning In Test Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot_multiple_bar(df_perform=test_perform_before,target_folder=target_folder,column=['pred_prob_p','pred_prob_s'],title=title,step=0.05,min_value=0,max_value=1)

In [ ]:
filename = '[24]_mixed_probability_distribution_test_data_after.png'
target_folder = join_by([model_path,'picture',filename],separator='\\')
title = 'P and S Arrival Picking Probability Distribution\nFrom Seisbench ' + pretrain.upper() + ' Pretrained ' + architecture + '\nAfter Fine Tuning In Test Data\n Dataset ' + sr + ' Hz and ' + wd + ' s Time Window'
make_group_barplot_multiple_bar(df_perform=test_perform_after,target_folder=target_folder,column=['pred_prob_p','pred_prob_s'],title=title,step=0.05,min_value=0,max_value=1)

# Performance Matrix

In [ ]:
train_before = make_row_assessment(df_perform=train_perform_before,row_name='Train Before Fine-Tuning')
train_after = make_row_assessment(df_perform=train_perform_after,row_name='Train After Fine-Tuning') 
test_before = make_row_assessment(df_perform=test_perform_before,row_name='Test Before Fine-Tuning')
test_after = make_row_assessment(df_perform=test_perform_after,row_name='Test After Fine-Tuning')
df_assessment = pd.concat([train_before,train_after,test_before,test_after])
df_assessment

In [ ]:
filename = 'performance_summary.csv'
target_folder = join_by([model_path,'performance',filename],separator='\\')
df_assessment.to_csv(target_folder,index=True)